# Creating a web app to deploy the model

In [ ]:
!pip install wandb
!pip install onnxruntime
!pip install -q gradio 

     |████████████████████████████████| 1.7 MB 5.3 MB/s 
     |████████████████████████████████| 140 kB 48.8 MB/s 
     |████████████████████████████████| 180 kB 44.4 MB/s 
     |████████████████████████████████| 97 kB 6.1 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=aac973e0303ccdf7a601092562369b053d11ccef14a7c2626ed85be8d09efbd6
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=ba78cf5757007a3d3db016a6423e0e9cb542d9d254e8a888bd3874ea5d012cf6
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools
     |████████████████████████████████| 4.8 MB 5.4 MB/s 
     |████████████████████████████████| 979 kB 5.5 MB/s 
     |███████████████████████

In [ ]:
import os
import wandb
import pandas as pd
import numpy as np
import onnxruntime as rt
from gradio import gradio as gr

## Initialize Weights and Biases

In [ ]:
os.environ["WANDB_API_KEY"] = "f18186cbea88a0af7b8845f2b0f9033603152679"

In [ ]:
run = wandb.init(project='employee_attrition')

wandb: Currently logged in as: krishnak20 (use `wandb login --relogin` to force relogin)


## Download the model and load it

In [ ]:
artifact_dir

'./artifacts/XGBoost_UserCar:v0'

In [ ]:
!ls -al  artifacts//run-88glsqdu-learning_curve:v0

ls: cannot access 'artifacts//run-88glsqdu-learning_curve:v0': No such file or directory


In [ ]:
import wandb
run = wandb.init()
artifact = run.use_artifact('krishnak20/employee_attrition/XGBoost_UserCar:v0', type='model')
artifact_dir = artifact.download()

http://onnx.ai/sklearn-onnx/auto_examples/plot_complex_pipeline.html#example-complex-pipeline

## Columns

In [ ]:
x_columns = ['Age', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField',
       'EnvironmentSatisfaction', 'Gender', 'HourlyRate', 'JobInvolvement',
       'JobLevel', 'JobRole', 'JobSatisfaction', 'MaritalStatus',
       'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'OverTime',
       'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole',
       'YearsSinceLastPromotion', 'YearsWithCurrManager']

cat_features = ['BusinessTravel', 'Department','Education','EnvironmentSatisfaction',
                'EducationField','Gender','JobInvolvement','JobLevel','JobRole','JobSatisfaction',
                'MaritalStatus','PerformanceRating','RelationshipSatisfaction',
                'StockOptionLevel','WorkLifeBalance','OverTime']

num_features = list(set(x_columns) - set(cat_features))                              

## Implement the predict() function

The followng inputs need to be supplied:

['KM_Driven', 'Fuel_Type', 'age',
'Transmission', 'Owner_Type', 'Seats',
'make', 'mileage_new', 'engine_new', 'model',
'power_new', 'Location']

In [ ]:
def predict_price(Age, Education, MonthlyIncome,
                  PerformanceRating, TotalWorkingYears, YearsSinceLastPromotion, 
                  ):

    inputs_dict = {'Education' : Education, 
              'MonthlyIncome': MonthlyIncome, 
              'Age': float(Age), 
              'PerformanceRating': PerformanceRating, 
              'TotalWorkingYears': TotalWorkingYears, 
              'YearsSinceLastPromotion': float(YearsSinceLastPromotion), 
              }

    df = pd.DataFrame(inputs_dict, index = [0])
    print(df)

    inputs = {c: df[c].values for c in df.columns}
    for c in num_features:
        inputs[c] = inputs[c].astype(np.float32)
    for k in inputs:
        inputs[k] = inputs[k].reshape((inputs[k].shape[0], 1))            
  
    sess = rt.InferenceSession(artifact_dir + '/usedcar_xgboost.onnx')
    pred_onx = sess.run(None, inputs)

    predicted_price = float(pred_onx[0][0,0])
    return {f'Expected sale price of the car is: INR {np.round(predicted_price, 2)} lakhs' }

## Implement UI

In [ ]:
YearsSinceLastPromotion = gr.inputs.Slider(minimum=1, maximum=10, label="Years Since Promotion")
MonthlyIncome = gr.inputs.Slider(minimum=0, maximum=4000000, label="Monthly Income")
Age = gr.inputs.Slider(minimum=1, maximum=70, label="Age")
PerformanceRating = gr.inputs.Slider(minimum=0, maximum=5, label="Performance Rating)")
TotalWorkingYears = gr.inputs.Dropdown(list(range(1, 20)), default="1", label="Total Working Years ")
Education = gr.inputs.Slider(minimum=0, maximum=5, label="Education")


gr.Interface(predict_price, [YearsSinceLastPromotion,MonthlyIncome, Age,PerformanceRating,TotalWorkingYears, Education], "text", live=False).launch(debug=True);

KeyboardInterrupt: ignored